<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Read-in-accession-ID-lists-created-from-initial-data-splitting" data-toc-modified-id="Read-in-accession-ID-lists-created-from-initial-data-splitting-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Read in accession ID lists created from initial data splitting</a></span></li><li><span><a href="#Explore-labeled-dataframe" data-toc-modified-id="Explore-labeled-dataframe-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Explore labeled dataframe</a></span><ul class="toc-item"><li><span><a href="#funtion:-print_row_detail" data-toc-modified-id="funtion:-print_row_detail-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>funtion: <code>print_row_detail</code></a></span></li></ul></li><li><span><a href="#Exploring-tags-that-match-any-of-the-regex-patterns" data-toc-modified-id="Exploring-tags-that-match-any-of-the-regex-patterns-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Exploring tags that match any of the regex patterns</a></span><ul class="toc-item"><li><span><a href="#Patterns-to-flag-candidate-paragraphs" data-toc-modified-id="Patterns-to-flag-candidate-paragraphs-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Patterns to flag candidate paragraphs</a></span></li><li><span><a href="#Tag-Exploration" data-toc-modified-id="Tag-Exploration-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Tag Exploration</a></span><ul class="toc-item"><li><span><a href="#Example-1:-Searching-for-block-elements-adjacent-to-Employee-headings" data-toc-modified-id="Example-1:-Searching-for-block-elements-adjacent-to-Employee-headings-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Example 1: Searching for block elements adjacent to <code>Employee</code> headings</a></span></li><li><span><a href="#Example-2:-An-employee-count-found-before-the-&quot;correct&quot;-one" data-toc-modified-id="Example-2:-An-employee-count-found-before-the-&quot;correct&quot;-one-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Example 2: An employee count found before the "correct" one</a></span></li><li><span><a href="#Example-3:-Regex-matches-do-not-include-employee-number-because-of-&lt;span/&gt;-elements" data-toc-modified-id="Example-3:-Regex-matches-do-not-include-employee-number-because-of-<span/>-elements-3.2.3"><span class="toc-item-num">3.2.3&nbsp;&nbsp;</span>Example 3: Regex matches do not include employee number because of <code>&lt;span/&gt;</code> elements</a></span></li></ul></li></ul></li><li><span><a href="#Test-regex" data-toc-modified-id="Test-regex-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Test regex</a></span><ul class="toc-item"><li><span><a href="#function:-check_regex_match" data-toc-modified-id="function:-check_regex_match-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>function: check_regex_match</a></span></li></ul></li><li><span><a href="#Write-functions-to-flag-paragraphs" data-toc-modified-id="Write-functions-to-flag-paragraphs-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Write functions to flag paragraphs</a></span><ul class="toc-item"><li><span><a href="#Find-candidate-paragraphs" data-toc-modified-id="Find-candidate-paragraphs-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Find candidate paragraphs</a></span></li><li><span><a href="#Create-dataframes-and-save-as-CSVs" data-toc-modified-id="Create-dataframes-and-save-as-CSVs-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Create dataframes and save as CSVs</a></span></li></ul></li><li><span><a href="#Debug-odd-findings-to-refine-regex-or-code" data-toc-modified-id="Debug-odd-findings-to-refine-regex-or-code-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Debug odd findings to refine regex or code</a></span><ul class="toc-item"><li><span><a href="#Explore-dataframe-of-paragraphs-for-training-data" data-toc-modified-id="Explore-dataframe-of-paragraphs-for-training-data-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Explore dataframe of paragraphs for training data</a></span></li></ul></li></ul></div>

# Retrieve candidate paragraphs from HTML files

## Read in accession ID lists created from initial data splitting

In [19]:
import pandas as pd

train_accession_ids = pd.read_csv('../data/train_accession_ids.csv', names=['acc_id'])['acc_id'].tolist()
val_accession_ids = pd.read_csv('../data/val_accession_ids.csv', names=['acc_id'])['acc_id'].tolist()

labeled_df = pd.read_excel('../data/train_val_employee_count_paragraphs.xlsx')
labeled_train_df = labeled_df.loc[labeled_df.split == 'train', :].copy()
labeled_val_df = labeled_df.loc[labeled_df.split == 'val', :].copy()
subset_df = pd.read_excel('../data/subset_employee_count_paragraphs.xlsx')

Since I already have the files, I can just loop through a directory to aquire the file paths, file names, and filing IDs (which are in the file name). The CIK numbers are useful for looking up additional filings on the [EDGAR website](https://www.sec.gov/cgi-bin/srch-edgar).

In [9]:
#filings_dir = PurePath('../employee_filings/')
#html_path_list = [PurePath(os.getcwd()).joinpath(file).as_posix() for file in glob.iglob("/*.".join([filings_dir.as_posix(), "html"]))]

#from path import Path, getcwdu
import glob
import os
from pathlib import PurePath
filing_suffix, filings_dir = (".html",  PurePath('../employee_filings/').as_posix() + "/")
html_path_list = [PurePath(os.getcwd()).joinpath(file).as_posix() for file in glob.iglob("*".join([filings_dir, filing_suffix]))]
html_file_list = [PurePath(path).name for path in html_path_list]

full_accession_ids = [PurePath(path).stem for path in html_path_list]
full_cik_nbrs = [x.split(sep='-')[0] for x in full_accession_ids]

# Create list of paths for subset_df, based on accession_id
subset_path_list = [PurePath(os.getcwd()).joinpath(filings_dir).joinpath(file) for file in html_file_list if PurePath(file).stem in subset_df.accession_number.unique().tolist()]


There are 30 files in the train + validation sets for which there are no labeled employee counts. Thus, these are the potential true negatives. 

In [20]:
print("Total files in train + validation subsets: ", len(full_accession_ids))
print("Files with employee count information, according to the golden dataset: ", 
      len(labeled_df.accession_number.unique().tolist()))
print("Total files in training set: ", len(train_accession_ids))
print("Training files with employee count information, according to the golden dataset: ", 
      len(labeled_train_df.accession_number.unique().tolist()))
print("Total files in validation set: ", len(val_accession_ids))
print("Training files with employee count information, according to the golden dataset: ", 
      len(labeled_val_df.accession_number.unique().tolist()))

Total files in train + validation subsets:  2222
Files with employee count information, according to the golden dataset:  2192
Total files in training set:  1667
Training files with employee count information, according to the golden dataset:  1641
Total files in validation set:  556
Training files with employee count information, according to the golden dataset:  551


## Explore labeled dataframe

### funtion: `print_row_detail`

This helper function is useful for inspecting the data.

In [5]:
def print_row_detail(df=subset_df, nrow=10, header_list = ['ticker', 'accession_number' ],
                    detail_list = ['data_key_friendly_name', 'data_value', 'reported_units', 'text', 'paragraph_text'],
                    sortby=['accession_number', 'data_key_friendly_name'], 
                     ascending=True):
    """Print nrows of df values for columns in detail_list, sequentially."""
    df_sorted = df.sort_values(sortby, ascending=ascending).reset_index()
    nrow = min(len(df_sorted), nrow)
    for i in range(0, nrow):
        for h in header_list:
            print('-'*35  + ' ' +  str(df_sorted[h][i]) + ' ' + '-'*35)
        for d in detail_list:
            print(d + '  :' + str(df_sorted[d][i]))
            print('')

print_row_detail(df=subset_df, nrow=20)

Only 15 files are included in the subset_df

In [8]:
len(subset_df.accession_number.unique())

15

In [9]:
subset_df.groupby('data_key_friendly_name').text.value_counts()

data_key_friendly_name  text                  
Full-Time Employees     full-time                 5
                        full-time Team            1
Other Employees         employees                 5
                        employed                  4
                        Total                     1
                        temporary                 1
Part-Time Employees     part-time                 2
                        part-time Team Members    1
Name: text, dtype: int64

## Exploring tags that match any of the regex patterns

### Patterns to flag candidate paragraphs

Modular components somestimes make it easier to organize (and update) regular expressions.

In [28]:
import re

# Initial components
employee_terms = "(associates|employees|full[ -]time[ -]equivalent(s)?|staff|team members|workers)"
person_terms = "(individuals|people|persons)" # These need additional cues
workforce_terms = "(((employee|employment|head|personnel|staff|worker|workforce) (count(s)|level(s)|total(s))+)|(head-count|headcount|workforce))"
employee_type_terms = "(full time|full-time|permanent|part time|part-time|regular|seasonal|temporary|total)"

numeral_pat = "(([0-9]{1,3},)*[0-9]{1,3}([.][0-9])?)" # Include numerals, requiring comma separation when appropriate, and allowing for decimals. 
rel_qualifiers = "(a total of|approximately|in aggregate|in total|(an|the) equivalent of|total)"

space_pat = "( |\n)" # In html, a space is often missing if the paragraph continues on the next line

magnitude_words = "(hundred|thousand|million|billion)" 
num_words = "(one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen)"

# Composites to allow numerals and/or number words with relative modifiers
num_pat = "".join(["((",numeral_pat, "|", num_words, ")(", space_pat, magnitude_words, ")*", ")"]) 
num_emps_pat = "".join([num_pat, space_pat, "(", rel_qualifiers, space_pat, ")*",  "(", employee_type_terms, space_pat, ")*"])

# Actual patterns to be used, named by the format they're meant to capture
number_employees_pat = "".join([num_emps_pat, employee_terms]) # A number followed by an employee term, allowing for qualifiers
employed_num_pat = "".join(["employ((ed|s)?)?", space_pat, "(", rel_qualifiers, space_pat, ")*", 
                        num_emps_pat]) # The verb employed, followed by a number
emp_type_emp_term_pat = "".join([employee_type_terms, space_pat, employee_terms]) # Part or full-time term, followed by an employee term
employed_end_span_pat = "".join(["employed(", space_pat, rel_qualifiers, ")*$"]) # The actual number is often cut off by a <span/> element in html
span_start_employees_pat = "".join(["^", "(", rel_qualifiers, space_pat, ")*",  
                                   "(", employee_type_terms, space_pat, ")*", 
                                   employee_terms]) # Employee terms at the beginning of a span

emp_pat_list = [number_employees_pat, employed_num_pat, emp_type_emp_term_pat, 
                employed_end_span_pat, span_start_employees_pat, workforce_terms] # Workforce terms just need a broad net
emp_pats = [re.compile(x, re.I) for x in emp_pat_list]

print_row_detail(labeled_train_df[labeled_train_df.paragraph_text.str.contains(re.compile("workforce"))])

Regex for identifying "block" html elements - useful for finding actual paragraph boundaries, or the first set of *n* paragraphs or tables immediately after a secion heading

In [30]:
block_re = re.compile(r"^(p|div|table)$")

### Tag Exploration

In [32]:
from bs4 import BeautifulSoup as bs

#### Example 1: Searching for block elements adjacent to `Employee` headings

In [61]:
with open(subset_file_list[2], encoding="utf8") as file: 
            file1_html = file.read()
            soup1 = bs(file1_html, 'lxml')
soup1_emp_count = soup1.find_all(string=[emp_pats])

print("Accession number:", subset_file_list[2].stem)
print("Matches:", len(soup1_emp_count))

emp1_tag = soup1_emp_count[0]
print("First match is: ", emp1_tag)
print("Type of first match: ", type(emp1_tag))
print("Tag parent is:", emp1_tag.parent) # The match is a string within a tag; the parent is the actual tag

Accession number: 0001069183-17-000042
Matches: 9
First match is:  Employees
Type of first match:  <class 'bs4.element.NavigableString'>
Tag parent is: <div class="c92">Employees</div>


The first tag is a section header, so I'm now interested in its siblings. 

In [41]:
for i, v in enumerate(emp1_tag.parent.find_next_siblings(block_re, limit=6)):
    print("Sibling number", i+1, ":")
    print("==" * 30)
    print(v)

Sibling number 1 :
<div class="c88"><span class="c16">As of December 31,</span> <span class="c16">2016</span><span class="c16">,
we had</span> <span class="c16">699</span> <span class="c16">full-time employees and</span> <span class="c16">202</span> <span class="c16">temporary employees. The breakdown of our full-time employees by department
is as follows:</span> <span class="c16">175</span> <span class="c16">direct manufacturing employees and</span>
<span class="c16">524</span> <span class="c16">administrative and manufacturing support employees. Of
the</span> <span class="c16">524</span> <span class="c16">administrative and manufacturing support
employees,</span> <span class="c16">213</span> <span class="c16">were involved in sales, marketing,
communications and training. Of the</span> <span class="c16">202</span> <span class="c16">temporary employees,
more than</span> <span class="c16">92%</span> <span class="c16">worked in direct manufacturing roles. Our
employees are not covered b

#### Example 2: An employee count found before the "correct" one

In [56]:
with open(subset_file_list[12], encoding="utf8") as file: 
            file2_html = file.read()
            soup2 = bs(file2_html, 'lxml')
soup2_emp_count = soup2.find_all(string=[emp_pats])

print("Accession number:", subset_file_list[12].stem)
print("Matches:", len(soup2_emp_count))

Accession number: 0001558370-17-001556
Matches: 7


In [57]:
print_row_detail(subset_df[subset_df.accession_number == '0001558370-17-001556'])

----------------------------------- AAOI -----------------------------------
----------------------------------- 0001558370-17-001556 -----------------------------------
data_key_friendly_name  :Full-Time Employees

data_value  :2776

reported_units  :ones

text  :full-time

paragraph_text  :As of December 31, 2016, we employed 2,776 full-time employees, of which 31 held Ph.D. degrees in a science   or engineering field. Of our employees, 287 are located in the U.S., 1,218 are located in Taiwan and 1,271   are located in China. None of our employees are represented by any collective bargaining agreement, but   certain employees of our China subsidiary are members of a trade union. We have never suffered any work   stoppage as a result of an employment related strike or any employee related dispute and believe that we have   satisfactory relations with our employees



In this example, the correct count is below an `Employees` section name. 

In [96]:
newline_pat = re.compile(r"([0-9a-zA-Z.(),])\n([0-9a-zA-Z.(),])") # To replace newline characters that have nothing to do with document structure
for i, v in enumerate(soup2_emp_count[:3]):
    print("Match number", i+1, ":")
    print("==" * 30)
    print(re.sub(newline_pat, r"\1 \2", v))

Match number 1 :
As of December 31, 2016, we had a total of 276 employees working in the R&D department, including 16 with Ph.D. degrees. We continue to recruit talented engineers to further enhance our research and development capabilities. We have research and development departments in our facilities in Texas, Georgia, China and Taiwan. Our research and development teams collaborate on joint projects, and by co-locating with our manufacturing operations enable us to achieve an efficient cost structure and improve our time to market.
Match number 2 :
Employees
Match number 3 :
As of December 31, 2016, we employed 2,776 full-time employees, of which 31 held Ph.D. degrees in a science or engineering field. Of our employees, 287 are located in the U.S., 1,218 are located in Taiwan and 1,271 are located in China. None of our employees are represented by any collective bargaining agreement, but certain employees of our China subsidiary are members of a trade union. We have never suffered 

A "heading" should be in a header tag, but it's not. 

In [47]:
soup2_emp_count[1].parent.parent

<p class="c75"><span class="c58">Employees</span></p>

#### Example 3: Regex matches do not include employee number because of `<span/>` elements

[SEC filing link](https://www.sec.gov/Archives/edgar/data/706688/000070668817000030/a10k4q2016.htm)

In [101]:
with open(subset_file_list[0], encoding="utf8") as file: 
            file3_html = file.read()
            soup3 = bs(file3_html, 'lxml')
soup3_emp_count = soup3.find_all(string=[emp_pats])

print("Accession number:", subset_file_list[0].stem)
print("Matches:", len(soup3_emp_count))

Accession number: 0000706688-17-000030
Matches: 13


This is a clear example in which the regex failed to actually match the string containing the number of employees; the employee count should be right in front of `'employees.'` in Match 2.

In [97]:
for i, v in  enumerate([re.sub(newline_pat, r"\1 \2", x) for x in  soup4_emp_count[:3]]):
    print("Match number", i+1, ":")
    print(re.sub(newline_pat, r"\1 \2", v))
    print("==" * 30)

Match number 1 :
Employees
Match number 2 :
employees. None of our employees are covered by a collective bargaining agreement and we believe that our relations with employees are good.
Match number 3 :
Our workforce is comprised primarily of employees who work on an hourly basis. To grow our operations and meet the needs and expectations of our customers, we must attract, train, and retain a large number of hourly associates, while at the same time controlling labor costs. These positions have historically had high turnover rates, which can lead to increased training, retention and other costs. In certain areas where we operate, there is significant competition for employees, including from retailers and the restaurant industries. The lack of availability of an adequate number of hourly employees, or our inability to attract and retain them, or an increase in wages and benefits to current employees could adversely affect our business, results of operations, cash flows and financial con

To verify, I'll use a helper function to show characters before and after a substring from the results. 

In [108]:
def show_str_context(search_str: str, in_string: str, char_before: int=500, char_after: int=2000, match_num: int=0):
    """Return characters before and after a given string."""
    sub_idx = in_string.find(search_str) 
    len_offset = len(search_str)
    if match_num:
        idx = sub_idx
        for i in range(match_num):
            idx = in_string.find(search_str, idx + len_offset)
        sub_idx = idx
    context_beg = max(sub_idx - char_before, 0)
    context_end = sub_idx + len_offset + char_after
    return in_string[context_beg:context_end]   

search_str = "employees are covered by a collective bargaining agreement and we believe that our relations with employees are"
print(re.sub(newline_pat, r"\1 \2",
       show_str_context(search_str, file4_html, char_before=200, char_after=200)))

<span class="c21">At</span> <span class="c21">December 31, 2016</span><span class="c21">, the Company had approximately</span> <span class="c21">11,500</span> <span class="c21">employees. None of our employees are covered by a collective bargaining agreement and we believe that our relations with employees are good.</span></div>
<div><a name="s0F8F0C9941555F6EB10D590A6566D4FF" id="s0F8F0C9941555F6EB10D590A6566D4FF"></a></div>
<div class="c59">Available Information</div>
<div class="c61"><span class="c21">W


## Test regex

###  function: check_regex_match

In [139]:
def check_regex_match(pattern, text_list):
    for idx, s in enumerate(text_list):
        mo = re.search(pattern, s)
        if mo:
            ms = mo.span()[1]
            print("------    " + str(idx) + "   Matched!    -----")
            print('str length  :' + str(len(s)) + '    match span  :' + str(ms))
            print(s[:ms])
            print(s[ms:])
            print(re.search(pattern, s))
        else:
            print("------    " + str(idx) + "  NO MATCH    -----")
            print(s)

## Write functions to flag paragraphs

notes:
- if r"^Employees$" matches, find the next block element that matches another pattern 
- Include tables as "matches" when siblings of Employees block elements

Need regex that will capture heading tags

In [135]:
head_block_re = re.compile(r"^(p|div|h[1-6])$") # Identify block-level elements, aside from tables
emp_head_pre = "((([0-9a-z](([.][0-9a-z])|([0-9a-z][.]))*[0-9a-z.]?)|Full-Time|Our|Number of|Total) ?)?"
emp_head_terms = "(Associates|Employees|Headcount|Personnel|Team Members|Staff|Workforce)"
#emp_head_raw = re.compile(r"[>]((Our|Number of|Total) )?(Associates|Employees|Headcount|Personnel|Team Members|Workforce)[<]", re.I)
#emp_head = re.compile(r"^((Our|Number of) )?(Associates|Employees|Headcount|Personnel|Team Members|Workforce)$", re.I)
emp_head_raw = re.compile("r" + "".join(["[>]", emp_head_pre, emp_head_terms, "([.:])?[<]"]), re.I)
emp_head = re.compile("".join(["^",   emp_head_pre, emp_head_terms, "([.:])?$"]), re.I)

Testing the regex for different employee heading formats:

In [140]:
emp_head_test_list = ["10 Employees", "D.EMPLOYEES", "8.3.18 Employees", "IX. Employees"]
check_regex_match(emp_head, emp_head_test_list)

------    0   Matched!    -----
str length  :12    match span  :12
10 Employees

<_sre.SRE_Match object; span=(0, 12), match='10 Employees'>
------    1   Matched!    -----
str length  :11    match span  :11
D.EMPLOYEES

<_sre.SRE_Match object; span=(0, 11), match='D.EMPLOYEES'>
------    2   Matched!    -----
str length  :16    match span  :16
8.3.18 Employees

<_sre.SRE_Match object; span=(0, 16), match='8.3.18 Employees'>
------    3   Matched!    -----
str length  :13    match span  :13
IX. Employees

<_sre.SRE_Match object; span=(0, 13), match='IX. Employees'>


Lists to accumulate flagged items; later turned into dataframes

### Find candidate paragraphs

In [50]:
acc_id_list = [] ; para_list_orig = [] ;  tag_list = [];
emp_head_list = []; emp_head_first_list = [];
tbl_acc_id_list = [] ; tbl_tag_list = []; 

When retaining tags from a document parsed by BeautifulSoup, it's important to simply copy or extract the tag you need. This gives you all descendants of the tag without keeping the entire parse tree in memory.

In [ ]:
import copy

This part takes some time, so it's ideal to split the work and run in parallel (even four notebooks running locally cuts the time from over 4 hours to a little over 1 hour - the task is mainly CPU bound). 

In [51]:
for i, fl in enumerate(html_path_list):
    acc_id = PurePath(fl).stem.replace('.html', '')
    tag_set = set();
    with open(fl, encoding="utf8") as file: 
        file_html = file.read()
        soup = bs(file_html, 'lxml')
#        emp_head_flag = False
#        emp_head_first_match = False
        if re.search(emp_head_raw, file_html):
            for ihead, hblock in enumerate(soup.body.find_all(string=emp_head, limit=4)):
                try:
                    emp_head_tag = hblock.find_parent(name=head_block_re)
                    if emp_head_tag.name != 'table' and emp_head_tag.find_parent('table') == None:
                        emp_head_matched = False
            #            print(emp_head_tag.name) ;print(emp_head_tag)
                        for i2, block in enumerate(emp_head_tag.find_next_siblings(block_re, limit=6)):
                            #print('Block sibling number: ' + str(i2))
                            if block.find(string=[emp_pats]) != None and block.name != 'table':
                                block_tag = copy.copy(block)
                                if block_tag not in tag_set:
                                    acc_id_list.append(acc_id) 
                                    tag_list.append(block_tag)
                                    para_list_orig.append(block_tag.get_text())
                                    tag_set.add(block_tag)
                                    emp_head_list.append(True)
                                    if not emp_head_matched:
                                        emp_head_flag = True
                                        emp_head_matched = True
                                        emp_head_first_list.append(True)
                                    else:
                                        emp_head_first_list.append(False)
                            if block.find('table') != None:
            #                    print('Found table match!')
                                tbl_block_tag = copy.copy(block)
                                if tbl_block_tag not in tag_set:
                                    tbl_acc_id_list.append(acc_id) 
                                    tbl_tag_list.append(tbl_block_tag)
                                    tag_set.add(tbl_block_tag)
                except:
                    continue
                        #tbl_df = pd.read_html(block.find('table').prettify(), tupleize_cols=True)[0].dropna(axis=1,how='all')
    #                    print(tbl_df); print(tbl_df.info()); print(block)
#        else:
#            print('No Employees header')
        soup_emp_count = soup.body.find_all(string=[emp_pats])
        soup_emp_paras = [x.find_parent(name=block_re) for x in soup_emp_count]
        soup_emp_paras = [x for x in soup_emp_paras if x != None]
        for i2, block in enumerate(soup_emp_paras):
#                print('Para number: ' + str(i2)); print(block)
            block_tag = copy.copy(block)
            if block_tag not in tag_set:
                if block.find('table') != None:
                    tbl_acc_id_list.append(acc_id) 
                    tbl_tag_list.append(block_tag)
                    tag_set.add(block_tag)
                else:
                    acc_id_list.append(acc_id) 
                    tag_list.append(block_tag)
                    tag_set.add(block_tag)
                    para_list_orig.append(block_tag.get_text())
                    emp_head_list.append(False)
                    emp_head_first_list.append(False)
#emp2_tag.find_next_siblings(block_re, limit = 1, string=False)[0].find(string=[emp_pats]).parent

In [52]:
print(len(set(acc_id_list)))
print(len(acc_id_list))
print(len(para_list_orig))
print(len(set(para_list_orig)))
print(len(tag_list))
print(len(emp_head_list))
print(len(emp_head_first_list))
print(len(tbl_acc_id_list))
print(len(tbl_tag_list))

555
3775
3775
3237
3775
3775
3775
57
57


### Create dataframes and save as CSVs

In [53]:
#### Make dataframe
tbl_html_df = pd.DataFrame(data = { 'acc_id': tbl_acc_id_list, 'tbl_html': tbl_tag_list, 'split' : 'train' })
tbl_html_df.loc[tbl_html_df.acc_id.isin(val_accession_ids),'split'] = 'val'

In [54]:
tbl_html_df.loc[:,'idx'] = tbl_html_df.index.values

tbl_html_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 4 columns):
acc_id      57 non-null object
split       57 non-null object
tbl_html    57 non-null object
idx         57 non-null int64
dtypes: int64(1), object(3)
memory usage: 1.9+ KB


In [55]:
#### Write to csv for later use
tbl_html_df.to_csv('data/tbl_html_df_1.csv')

In [56]:
paragraph_input_dict = {'acc_id' : acc_id_list, 
                        'para_text' : [p.replace('\n', ' ').strip().replace(' ,', ',') for p in para_list_orig],
                        'len' : [len(p) for p in para_list_orig], 
                        'emp_header' : emp_head_list,
                        'first_emp_head_block' : emp_head_first_list,
                       'para_text_orig' : para_list_orig, 
                        'para_tag' : tag_list, 
                       'split' : 'train', 
                       'label' : 0 }
#paragraph_input_df['para_text'] = paragraph_input_df.para_text_orig.replace('\n', ' ')
p_columns = ['acc_id', 'para_text', 'len', 'emp_header', 'first_emp_head_block', 'para_text_orig',
              'para_tag', 'split', 'label']

paragraph_input_df = pd.DataFrame(paragraph_input_dict, columns=p_columns)


paragraph_input_df.loc[paragraph_input_df.acc_id.isin(val_accession_ids),'split'] = 'val'

train_df = paragraph_input_df[paragraph_input_df.split == 'train']

In [57]:
paragraph_input_df.loc[:,'idx'] = paragraph_input_df.index.values

paragraph_input_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3775 entries, 0 to 3774
Data columns (total 10 columns):
acc_id                  3775 non-null object
para_text               3775 non-null object
len                     3775 non-null int64
emp_header              3775 non-null bool
first_emp_head_block    3775 non-null bool
para_text_orig          3775 non-null object
para_tag                3775 non-null object
split                   3775 non-null object
label                   3775 non-null int64
idx                     3775 non-null int64
dtypes: bool(2), int64(3), object(5)
memory usage: 243.4+ KB


In [58]:
paragraph_input_df.head()

,acc_id,para_text,len,emp_header,first_emp_head_block,para_text_orig,para_tag,split,label,idx
0,0000003570-17-000052,"We had 911 full-time employees at January 31, ...",53,True,True,"We had 911 full-time\nemployees at January 31,...","<div class=""c93""><span class=""c14"">We had</spa...",val,0,0
1,0000003570-17-000052,Employees,9,False,False,Employees,"<div class=""c91"">Employees</div>",val,0,1
2,0000003570-17-000052,Our total operating costs and expenses increas...,778,False,False,Our total operating costs and expenses increas...,"<div class=""c93""><span class=""c14"">Our total o...",val,0,2
3,0000003570-17-000052,Blackstone Group disclosed that NCR Corporatio...,1826,False,False,Blackstone Group disclosed that NCR Corporatio...,"<div class=""c93""><span class=""c14"">Blackstone ...",val,0,3
4,0000004127-16-000068,"As of September 30, 2016, we employed approxim...",245,True,True,"As of September 30, 2016,\nwe employed approxi...","<div class=""c80""><span class=""c32"">As of</span...",train,0,4


In [59]:
paragraph_input_df.to_csv('data/paragraph_input_df_1.csv')

## Debug odd findings to refine regex or code

Code for printing results from paragraph search for a file

In [62]:
def print_parse_objects(file: str, use_acc_id: bool=True, acc_id_list = full_accession_ids, 
                       file_path_list = html_path_list):
    fl = file
    if use_acc_id:
        fl = html_path_list[full_accession_ids.index(fl)]        
        print('Index number: ')
        print(full_accession_ids.index(file))
    print('File path: ')
    print(fl)
    #for i, fl in enumerate(html_path_list[303:304]):
    with open(fl, encoding="utf8") as file: 
        file_html = file.read()
        soup = bs(file_html, 'lxml')
        emp_head_flag = False
        emp_head_first_match = False
        if re.search(emp_head_raw, file_html):
            for ihead, hblock in enumerate(soup.body.find_all(string=emp_head)):
                emp_head_tag = hblock.find_parent(name=head_block_re)

                if emp_head_tag != None and emp_head_tag.name != 'table' and emp_head_tag.find_parent('table') == None:
                    emp_head_matched = False
                    print(ihead); 
                    try:
                        print(emp_head_tag.name) 
                    except:
                        print('emp_head_tag has no name')
                    print(emp_head_tag)
                    for i2, block in enumerate(emp_head_tag.find_next_siblings(block_re, limit=6)):
                        print('Block sibling number: ' + str(i2))
                        if block.find(string=emp_pats) != None:
                            block_tag = copy.copy(block)
                            print('Found match!'); print(block_tag) ; print(block_tag.get_text())
                        if block.find('table') != None:
                            print('Found table match!')
                            tbl_block_tag = copy.copy(block)
                            tbl_df = pd.read_html(block.find('table').prettify())[0].dropna(axis=1,how='all')
                            print(tbl_df); print(tbl_df.info()); print(block)
        #else:
        print('No Employees header')
        soup_emp_count = soup.body.find_all(string=[emp_pats])
        soup_emp_paras = [x.find_parent(name=head_block_re) for x in soup_emp_count]
        soup_emp_paras = [x for x in soup_emp_paras if x != None]
        soup_emp_blocks = [x  for x in soup_emp_count if x.name in ['p', 'div']]
        for i2, block in enumerate(soup_emp_paras):
            block_tag = copy.copy(block)
            print('Para number: ' + str(i2)); 
            try:
                print('Original match name:  ' +  soup_emp_count[i2].name)
            except:
                print('Original match has no name')
            print('Original match:  ' +  soup_emp_count[i2])
            try:
                print('Block match name:  ' +  block_tag.name)
            except:
                print('Block match has no name')
            print('Block:  ')
            print(block)
            print(type(block))
            print('Block Text:  ')
            print(block.get_text().replace('\n', ' '))
        for i2, block in enumerate(soup_emp_blocks):
            print('Block number: ' + str(i2)); print(block)
#emp2_tag.find_next_siblings(block_re, limit = 1, string=False)[0].find(string=[emp_pats]).parent

In [77]:
print('Paragraphs: ' + str(len(acc_id_list)))
print('Files with paragraphs: ' + str(len(set(acc_id_list))))
print('Tables: ' + str(len(tbl_acc_id_list)))

Paragraphs: 8205
Files with paragraphs: 2182
Tables: 208


Code for investigating files for which no candidate paragraphs were identified

In [78]:
missed_ids = [x for x in full_accession_ids[:100] if x not in set(acc_id_list)]
# For investigating files with no candidate paragraphs if they are part of the training set
#[x for x in missed_ids if x in train_accession_ids]  

In [350]:
#find index of file for specific accession id
#[i for i, x in enumerate(html_path_list[:300]) if re.search(r"0000042316-17-000014", x)]

[89]

### Explore dataframe of paragraphs for training data

In [460]:
train_df.head(2)

,acc_id,para_text,len,emp_header,first_emp_head_block,para_text_orig,para_tag,split,label
2,0000004127-16-000068,"As of September 30, 2016, we employed approxim...",245,True,True,"As of September 30, 2016,\nwe employed approxi...","<div class=""c80""><span class=""c32"">As of</span...",train,0
3,0000004127-16-000068,EMPLOYEES,9,False,False,EMPLOYEES,"<div class=""c90"">EMPLOYEES</div>",train,0


In [79]:
train_df.describe()

,len,label
count,6170.000000,6170.0
mean,415.585575,0.0
std,475.612180,0.0
min,8.000000,0.0
25%,26.000000,0.0
50%,300.000000,0.0
75%,618.000000,0.0
max,5054.000000,0.0


In [490]:
train_df.len.value_counts().to_frame('len_counts').sort_index(ascending=False)

,len_counts
5054,1
4944,1
4868,1
4213,1
3972,1
3904,1
3878,1
3862,1
3795,1
3791,1


In [80]:
print_row_detail(df=train_df, nrow=10, header_list = ['acc_id'  ],
                    detail_list = ['len', 'emp_header', 'first_emp_head_block', 'para_text'],
                    sortby=['len', 'acc_id'], ascending=False)

----------------------------------- 0001193125-17-106055 -----------------------------------
len  :5054

emp_header  :False

first_emp_head_block  :False

para_text  :Current format     Previous format                             Millions of Euros       Millions of Euros    ASSETS   December  2015     December  2014      ASSETS   December  2015     December  2014              CASH, CASH BALANCES AT CENTRAL BANKS AND OTHER DEMAND DEPOSITS (1)   29,282    27,719     CASH AND BALANCES WITH CENTRAL BANKS   43,467    31,430    FINANCIAL ASSETS HELD FOR TRADING   78,326    83,258     FINANCIAL ASSETS HELD FOR TRADING   78,326    83,258    Derivatives   40,902    44,229     Loans and advances to credit institutions   -    -    Equity instruments   4,534    5,017     Loans and advances to customers   65    128    Debt securities   32,825    33,883     Debt securities   32,825    33,883    Loans and advances to central banks   -    -     Equity instruments   4,534    5,017    Loans and advances

In [68]:
print_parse_objects(file='0001047469-16-014916')

Index number: 
734
File path: 
c:/projects/DSBC/capstone/sec_employee_information_extraction/../employee_filings/0001047469-16-014916.html
No Employees header
Para number: 0
Original match has no name
Original match:  We currently have 21 employees, 18 of whom are located in Denver, Colorado, one who is located
in Zug, Switzerland, and two who are located in Toronto, Canada. Our employees are not subject to a labor
contract or a collective bargaining agreement. We consider our employee relations to be good.
Block match name:  p
Block:  
<p class="c15">We currently have 21 employees, 18 of whom are located in Denver, Colorado, one who is located
in Zug, Switzerland, and two who are located in Toronto, Canada. Our employees are not subject to a labor
contract or a collective bargaining agreement. We consider our employee relations to be good.</p>
<class 'bs4.element.Tag'>
Block Text:  
We currently have 21 employees, 18 of whom are located in Denver, Colorado, one who is located in Zug, 